In [25]:
import math

import pandas as pd
from pandas import DataFrame
from pandas import concat

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

import seaborn as sns
sns.set()

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from math import sqrt
from matplotlib import pyplot

from scipy.io import loadmat
import os
from os import path

import warnings
warnings.filterwarnings("ignore")

DATA_DIR = 'data/preprocessed'

np.random.seed(3)

In [26]:
# Using plotly + cufflinks in offline mode
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import cufflinks
cufflinks.go_offline(connected=True)

In [27]:
# recursive multi-step forecast with linear algorithms
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor

In [28]:
import re
Data = {}
for file in os.listdir(DATA_DIR):
    filename = os.fsdecode(file)
    if filename.endswith(".mat"):
        data = loadmat(path.join(DATA_DIR, filename))
        index = re.findall(r'\d+', filename)[-1]
        Data['electrode_'+str(index)] = np.squeeze(data['p1st'])

df = pd.DataFrame(Data)

In [29]:
df = df.T
print("The dataset contains {0} rows and {1} columns".format(df.shape[0], df.shape[1]))
df.head()

The dataset contains 103 rows and 1802762 columns


,0,1,2,3,4,5,6,7,8,9,...,1802752,1802753,1802754,1802755,1802756,1802757,1802758,1802759,1802760,1802761
electrode_1,41.716805,38.812707,35.041442,30.383131,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.788420,...,-19.207158,-19.262882,-18.675911,-17.487823,-15.763623,-13.582821,-11.026026,-8.168095,-5.070118,-1.778862
electrode_10,8.082549,11.133247,14.296853,17.412122,20.338028,22.955956,25.170391,26.907064,28.111660,28.743052,...,33.226267,23.681644,14.175155,5.087411,-3.229318,-10.462901,-16.353534,-20.704262,-23.390363,-24.367822
electrode_100,31.998051,33.929033,36.232118,38.845610,41.689640,44.676844,47.714919,50.709624,53.570229,56.209865,...,-60.975075,-58.528180,-54.766563,-49.793458,-43.776389,-36.936385,-29.537547,-21.874607,-14.252103,-6.968529
electrode_101,-37.003890,-39.624473,-41.209704,-41.773115,-41.354811,-40.018294,-37.845699,-34.934051,-31.393004,-27.339603,...,-0.425525,-1.024245,-1.689228,-2.386713,-3.115931,-3.916899,-4.872857,-6.102297,-7.746489,-9.956018
electrode_102,22.816080,20.600437,18.836262,17.521505,16.602531,15.978348,15.508288,15.026483,14.350623,13.297051,...,26.047437,26.329604,25.668915,24.330952,22.598718,20.752124,19.046779,17.703170,16.894929,16.738447


In [30]:
single_elec_df = df.loc[['electrode_1'],:]
print(single_elec_df.shape)
single_elec_df.head()

(1, 1802762)


,0,1,2,3,4,5,6,7,8,9,...,1802752,1802753,1802754,1802755,1802756,1802757,1802758,1802759,1802760,1802761
electrode_1,41.716805,38.812707,35.041442,30.383131,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.78842,...,-19.207158,-19.262882,-18.675911,-17.487823,-15.763623,-13.582821,-11.026026,-8.168095,-5.070118,-1.778862


In [31]:
# configure
n_window = 2**9
n_predict = 2**5
n_samples = 10000

In [32]:
def series_to_supervised_df(data, n_window=1, n_predict=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_window, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_predict):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [33]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_window, n_predict):
    # extract raw values
    raw_values = series.values
    raw_values = raw_values.reshape(len(raw_values), 1)
    # transform into supervised learning problem X, y
    supervised = series_to_supervised_df(raw_values, n_window, n_predict)
    # split into train and test sets
    return supervised

In [34]:
# prepare a list of ml models
def get_models(models=dict()):
    # linear models
    models['lr'] = LinearRegression()
    models['lasso'] = Lasso()
    models['ridge'] = Ridge()
    models['en'] = ElasticNet()
    models['huber'] = HuberRegressor()
#     models['lars'] = Lars()
    models['llars'] = LassoLars()
    models['pa'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3)
    models['ranscac'] = RANSACRegressor()
#     models['sgd'] = SGDRegressor(max_iter=1000, tol=1e-3)
    print('Defined %d models' % len(models))
    return models

In [35]:
# convert history into inputs and outputs
def series_to_supervised(history, n_window):
    X, y = list(), list()
    ix_start = 0
    # step over the entire history one time step at a time
    for i in range(len(history)):
        # define the end of the input sequence
        ix_end = ix_start + n_window
        # ensure we have enough data for this instance
        if ix_end < len(history):
            X.append(history[ix_start:ix_end])
            y.append(history[ix_end])
        # move along one time step
        ix_start += 1
    return array(X), array(y)

In [36]:
# create a feature preparation pipeline for a model
def make_pipeline(model):
    steps = list()
    # standardization
    steps.append(('standardize', StandardScaler()))
    # normalization
    steps.append(('normalize', MinMaxScaler()))
    # the model
    steps.append(('model', model))
    # create pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

In [37]:
# make a recursive multi-step forecast
def forecast_sequence(model, input_x, n_window, n_predict=1):
    yhat_sequence = list()
    input_seq = [x for x in input_x]
    for i in range(n_predict):
        # prepare the input data
        X = array(input_seq[-n_window:]).reshape(1, n_window)
        # make a one-step forecast
        yhat = model.predict(X)[0]
        # add to the result
        yhat_sequence.append(yhat)
        # add the prediction to the input
        input_seq.append(yhat)
    return yhat_sequence

In [38]:
# fit a model and make a forecast
def make_forecasts(model, history, n_window, n_predict):
    # prepare data
    train_x, train_y = series_to_supervised(history, n_window)
    # make pipeline
    pipeline = make_pipeline(model)
    # fit the model
    pipeline.fit(train_x, train_y)
    # predict the output, recursively
    yhat_sequence = forecast_sequence(pipeline, train_x[-1, :], n_window, n_predict)
    return yhat_sequence

In [39]:
def evaluate_forecasts(actual, forecasts, n_predict):
    scores = []
    forecasts_num = actual.shape[1]
    for i in range(n_predict):
        actual_samples = [row[i] for row in actual]
        predicted_samples = [forecast[i] for forecast in forecasts]
        rmse = sqrt(mean_squared_error(actual_samples, predicted_samples))
#         print('t+%d RMSE: %f' % ((i+1), rmse))
        scores.append(rmse)
    return scores

In [40]:
# evaluate a single model
def evaluate_model(model, train, test, n_window, n_predict, actual):
    # history is a list of batched data
    history = list(train.flatten())
    # walk-forward validation
    predictions = list()
    for i in range(len(test)-n_predict+1):
        # predict multi-step
        yhat_sequence = make_forecasts(model, history, n_window, n_predict)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next multi-step
        history.append(test[i, :])
        if i%500 == 0:
            print('evaluated ' + str(i) + ' samples out of ' + str(len(test)))
    # predictions is list of lists with shape 'test_size x n_predict'
    predictions = array(predictions)
    # evaluate predictions days for each multi-step
    scores = evaluate_forecasts(actual, predictions, n_predict)
    return (scores, actual, predictions)

## Data Observation

In [41]:
# prepare data
series = single_elec_df.T[:n_samples]
n_test = min(math.floor(0.2*len(series)), 10000)

supervised_df = prepare_data(series, n_window, n_predict)
print(supervised_df.shape)
supervised_df.head()

(9457, 544)


,var1(t-512),var1(t-511),var1(t-510),var1(t-509),var1(t-508),var1(t-507),var1(t-506),var1(t-505),var1(t-504),var1(t-503),...,var1(t+22),var1(t+23),var1(t+24),var1(t+25),var1(t+26),var1(t+27),var1(t+28),var1(t+29),var1(t+30),var1(t+31)
512,41.716805,38.812707,35.041442,30.383131,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.788420,...,-29.607517,-36.382068,-41.882933,-45.991190,-48.674418,-49.980838,-50.030888,-49.004865,-47.120562,-44.621989
513,38.812707,35.041442,30.383131,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.788420,-27.456232,...,-36.382068,-41.882933,-45.991190,-48.674418,-49.980838,-50.030888,-49.004865,-47.120562,-44.621989,-41.755972
514,35.041442,30.383131,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.788420,-27.456232,-38.565406,...,-41.882933,-45.991190,-48.674418,-49.980838,-50.030888,-49.004865,-47.120562,-44.621989,-41.755972,-38.753155
515,30.383131,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.788420,-27.456232,-38.565406,-49.873232,...,-45.991190,-48.674418,-49.980838,-50.030888,-49.004865,-47.120562,-44.621989,-41.755972,-38.753155,-35.817117
516,24.813376,18.310412,10.863951,2.491293,-6.757452,-16.788420,-27.456232,-38.565406,-49.873232,-61.096788,...,-48.674418,-49.980838,-50.030888,-49.004865,-47.120562,-44.621989,-41.755972,-38.753155,-35.817117,-33.117388


In [42]:
train_df, test_df = supervised_df.iloc[:-n_test,:], supervised_df.iloc[-n_test:,:] 

In [43]:
test_df.head()

,var1(t-512),var1(t-511),var1(t-510),var1(t-509),var1(t-508),var1(t-507),var1(t-506),var1(t-505),var1(t-504),var1(t-503),...,var1(t+22),var1(t+23),var1(t+24),var1(t+25),var1(t+26),var1(t+27),var1(t+28),var1(t+29),var1(t+30),var1(t+31)
7969,-6.480749,-5.722407,-4.667360,-3.321012,-1.696935,0.177308,2.265505,4.524492,6.902417,9.339648,...,-9.801603,-6.819594,-4.054144,-1.558873,0.604930,2.375511,3.697434,4.526073,4.827595,4.580679
7970,-5.722407,-4.667360,-3.321012,-1.696935,0.177308,2.265505,4.524492,6.902417,9.339648,11.776640,...,-6.819594,-4.054144,-1.558873,0.604930,2.375511,3.697434,4.526073,4.827595,4.580679,3.777484
7971,-4.667360,-3.321012,-1.696935,0.177308,2.265505,4.524492,6.902417,9.339648,11.776640,14.150922,...,-4.054144,-1.558873,0.604930,2.375511,3.697434,4.526073,4.827595,4.580679,3.777484,2.428167
7972,-3.321012,-1.696935,0.177308,2.265505,4.524492,6.902417,9.339648,11.776640,14.150922,16.398909,...,-1.558873,0.604930,2.375511,3.697434,4.526073,4.827595,4.580679,3.777484,2.428167,0.561502
7973,-1.696935,0.177308,2.265505,4.524492,6.902417,9.339648,11.776640,14.150922,16.398909,18.454614,...,0.604930,2.375511,3.697434,4.526073,4.827595,4.580679,3.777484,2.428167,0.561502,-1.772572


In [44]:
def create_actual(test, n_predict):
    res = []
    values = list(test.flatten())
    for j in range(len(values)-n_predict+1):
        res.append(values[j:j+n_predict])
    res = array(res)
    return res

## MAIN

In [ ]:
# split into train and test
dataset = series
train, test = dataset[:-n_test].values, dataset[-n_test:].values 

# prepare the models to evaluate
models = get_models()

# # create 
# multisteps = [str(i+1) for i in range(n_predict)]

# actual results to compare
actual = create_actual(test, n_predict)

graph_obj = {}

# evaluate each model
for name, model in models.items():
    print('Evaluate model ' + name)
    # evaluate and get scores
    scores, results, predictions = evaluate_model(model, train, test, n_window, n_predict, actual)
    graph_obj[name] = scores

graph_df = DataFrame(graph_obj)

In [46]:
graph_df.iplot(xTitle='#Prediction', yTitle='RMSE')